In [1]:
pip install arviz

     |████████████████████████████████| 1.6MB 4.9MB/s 
     |████████████████████████████████| 4.7MB 22.3MB/s 
     |████████████████████████████████| 808kB 35.4MB/s 
     |████████████████████████████████| 317kB 37.3MB/s 
  Found existing installation: xarray 0.15.1
    Uninstalling xarray-0.15.1:
      Successfully uninstalled xarray-0.15.1


In [2]:
import pystan
import numpy as np
import matplotlib.pyplot as plt
import arviz
import pickle
import cmdstanpy
import json
import pandas as pd

In [3]:
sm = pystan.StanModel(file='/content/drive/MyDrive/TFM-Crowd learning/Fitxers STAN/multinom_confusion_matrix.stan')

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_38a56093ba42788a25cd0fbe0f6a6a8f NOW.


# Example with random $\pi$ and $\tau$

In [4]:
pi = np.random.dirichlet((5, 1, 1), 3)
aux=pi[1,1];pi[1,1]=pi[1,0];pi[1,0]=aux
aux=pi[2,2];pi[2,2]=pi[2,0];pi[2,0]=aux
print(pi)

p=np.array([0.1,0.6,0.3])
n=1000
t=10
l=3
tc = np.random.choice(l,p=p,size=n)
x = np.zeros((n,l),dtype=np.int)
for i in range(n):
  x[i,:] = np.random.multinomial(t,pi[tc[i],:],size=1)
print(x)
prior_p = np.array([1,1,1],dtype=np.int)
prior_pi = np.array([1,1,1],dtype=np.int)
dades={'n':n,'x':x,'k': l,'prior_p':prior_p,'prior_pi':prior_pi}

[[0.64094493 0.3304241  0.02863098]
 [0.34039041 0.38766749 0.2719421 ]
 [0.07488878 0.22944495 0.69566627]]
[[0 3 7]
 [2 5 3]
 [1 2 7]
 ...
 [0 3 7]
 [3 5 2]
 [5 2 3]]


In [5]:
fit = sm.sampling(data=dades, iter=500, chains=4)

To run all diagnostics call pystan.check_hmc_diagnostics(fit)


In [6]:
pi_ini = np.eye(3)+0.05
pi_ini /= np.sum(pi_ini,axis=1)
print(pi_ini)
dades_inicials={'pi':pi_ini}

[[0.91304348 0.04347826 0.04347826]
 [0.04347826 0.91304348 0.04347826]
 [0.04347826 0.04347826 0.91304348]]


In [7]:
sm.optimizing(data=dades,init=dades_inicials)

OrderedDict([('p', array([0.5783812 , 0.12308913, 0.29852967])),
             ('pi', array([[0.32709816, 0.38920459, 0.28369725],
                     [0.62583956, 0.34036516, 0.03379528],
                     [0.078664  , 0.22174497, 0.69959104]])),
             ('log_tc', array([[ -7.4099142 , -24.25260179,  -3.4408917 ],
                     [ -3.44829144, -10.75113817,  -7.06492599],
                     [ -6.48514674, -22.54491449,  -3.37862179],
                     ...,
                     [ -7.4099142 , -24.25260179,  -3.4408917 ],
                     [ -3.30593878,  -7.83236537,  -9.25023635],
                     [ -3.96982594,  -8.92391314, -10.1739531 ]])),
             ('tc',
              array([1., 1., 1., 2., 3., 2., 1., 1., 1., 2., 3., 3., 3., 3., 1., 1., 2.,
                     1., 3., 3., 1., 1., 3., 1., 3., 3., 1., 2., 2., 1., 3., 1., 2., 1.,
                     2., 2., 1., 1., 2., 3., 3., 2., 3., 1., 2., 1., 3., 3., 1., 3., 1.,
                     1., 3., 1., 

# Code with JSON file

In [8]:
with open('/content/drive/MyDrive/TFM-Crowd learning/multinomial.json') as json_file:
    data = json.load(json_file)

In [10]:
dades={'Tasca': data['t_A'], 'Worker': data['w_A'], 'Anotació': data['ann'] }
df = pd.DataFrame(data=dades)
df

,Tasca,Worker,Anotació
0,0,14,2
1,0,13,2
2,0,4,2
3,0,2,2
4,0,7,1
...,...,...,...
9995,999,17,2
9996,999,2,2
9997,999,13,2
9998,999,14,2


In [13]:
anotacions=np.asarray(df['Anotació'])
anotacions.shape

(10000,)

In [14]:
x=np.zeros((1000,3),dtype=np.int)
aux1=0
aux2=0
for i in range(anotacions.shape[0]):
  aux2+=1
  if anotacions[i]==0:
    x[aux1][0]+=1
  elif anotacions[i]==1:
    x[aux1][1]+=1
  elif anotacions[i]==2:
    x[aux1][2]+=1
  if aux2==10:
    aux2=0
    aux1+=1
print(x)
print(x.shape)



[[ 0  1  9]
 [ 7  1  2]
 [ 1  2  7]
 ...
 [ 1  8  1]
 [ 1  0  9]
 [ 0  0 10]]
(1000, 3)


In [15]:
pi_ini = np.eye(3)+0.05
pi_ini /= np.sum(pi_ini,axis=1)
print(pi_ini)
dades_inicials={'pi':pi_ini}

[[0.91304348 0.04347826 0.04347826]
 [0.04347826 0.91304348 0.04347826]
 [0.04347826 0.04347826 0.91304348]]


In [18]:
n=x.shape[0]
l=x.shape[1]
prior_p = np.array([1,1,1])
prior_pi = np.array([1,1,1])
dades2={'n':n,'x':x,'k': l,'prior_p':prior_p,'prior_pi':prior_pi}

In [22]:
opt= sm.optimizing(data=dades2,init=dades_inicials)
opt

OrderedDict([('p', array([0.11447845, 0.29650916, 0.58901239])),
             ('pi', array([[0.68293363, 0.10402475, 0.21304162],
                     [0.21832192, 0.59436654, 0.18731154],
                     [0.09335238, 0.10484805, 0.80179957]])),
             ('log_tc', array([[-16.04431946, -14.50818964,  -2.47003565],
                     [ -4.30642971,  -9.85228862, -13.93985762],
                     [-12.01274909,  -9.61675029,  -3.07134048],
                     ...,
                     [-17.70019558,  -4.07470679, -16.66371382],
                     [-14.16255066, -15.50971518,  -2.58616643],
                     [-17.63004587, -17.96549771,  -2.73827425]])),
             ('tc',
              array([2., 1., 2., 3., 1., 3., 1., 1., 3., 3., 1., 2., 3., 3., 2., 3., 2.,
                     1., 3., 2., 2., 1., 3., 1., 1., 1., 1., 3., 3., 3., 1., 1., 3., 3.,
                     2., 1., 3., 3., 1., 2., 2., 1., 3., 2., 2., 1., 1., 2., 2., 2., 1.,
                     3., 3., 3., 